# Logic of MapReduce (under the Hadoop Streaming API)

This is a Jupyter Notebook running the bash (unix shell) kernel. Download the notebook file itself [here](https://burtmonroe.github.io/SoDA501/SplitApplyCombine_MapReduce/MapReduceLogic.ipynb).

See the broader "SplitApplyCombine" context here.

The Hadoop Streaming API executes a MapReduce functionality that allows you to write mappers and reducers in any language. 

The mapper has to be a script that reads input from the "standard input" and outputs (to "standard output") text representing (key,value) pairs. Each line of output emits a textual key and a textual value separated by a tab ("\t") character.

The reducer is a script that reads such (key,value) pairs and emits some (associative) summary of values by keys.

On the Hadoop system there are intermediate processors that "shuffle" the key-value pairs, sending pairs with particular reducers. This is taken care of by the Hadoop system automatically.

## Word Count with Python mapper and Python reducer

It's easiest to start with the canonical word counting task, which we'll do first in Python. 

### mapper.py

The mapper.py script reads text line by line, processes each line into words, and prints each word followed by a tab character, followed by the number 1, followed by an end of line.

Let's take a look at it first:

In [1]:
cat mapper.py

#!/usr/bin/env python
import sys, string

# Read lines from the "standard input" (sys.stdin).
#    sys.stdin is treated like a file object would be.
#    Unless you change it, this means "the console."
#    This could be from your keyboard or the output of
#    a command like "cat."

for line in sys.stdin:

    # lower-case, remove punctuation, strip extra whitespace
    line = line.lower()
    line = line.translate(str.maketrans('','',string.punctuation))
    line = line.strip()

    # split line into words (on spaces)
    words = line.split()

    # print (to standard output) key value pairs: "word 1"
    for word in words:
        print(word,1,sep="\t")


### reducer.py

The reducer.py script reads those pairs as an input, stores running totals in a dictionary, and then emits those.


In [2]:
cat reducer.py

#!/usr/bin/env python
import sys

# dictionary, keys will be words, values will be counts
wordcountdict = {}

# read key-value pair "word\tcount\n" from standard input
for line in sys.stdin:

    word, value = line.split('\t', 1)
    value = int(value)

    try:
        wordcountdict[word] = wordcountdict[word]+value
    except:
        wordcountdict[word] = value #new word

for word in wordcountdict.keys():
    print(word, wordcountdict[word], sep="\t")


### Input text: Trump's 2017 "State of the Union"

We'll use Donald Trump's 2017 Joint Address to Congress (the rough equivalent of the State of the Union in a President's first year) as a text input. Let's look at the top of that:


In [3]:
head trumpsotu2017.txt

Thank you very much. Mr. Speaker, Mr. Vice President, Members of Congress, the First Lady of the United States, and citizens of America: Tonight, as we mark the conclusion of our celebration of Black History Month, we are reminded of our Nation's path towards civil rights and the work that still remains to be done. Recent threats targeting Jewish community centers and vandalism of Jewish cemeteries, as well as last week's shooting in Kansas City, remind us that while we may be a nation divided on policies, we are a country that stands united in condemning hate and evil in all of its very ugly forms.
Each American generation passes the torch of truth, liberty, and justice in an unbroken chain, all the way down to the present. That torch is now in our hands, and we will use it to light up the world. I am here tonight to deliver a message of unity and strength, and it is a message deeply delivered from my heart. A new chapter of American greatness is now beginning. A new national pride is

### Text -> Mapper

One nice thing about this paradigm is that we can test our mappers and reducers locally. (In fact, that's all we'll do in this notebook.)

First we `cat` our input text (print it to "the console") and then "pipe" that `|` to `mapper.py`. We don't actually observe the printing of the text to the screen ... the pipe character tells the shell to treat the output of the thing on the left of the pipe as the input of the thing on the right of it.

Here we see the mapper turn the text into key-value pairs one at a time, in the order it sees them:


In [4]:
cat trumpsotu2017.txt | python mapper.py

thank	1
you	1
very	1
much	1
mr	1
speaker	1
mr	1
vice	1
president	1
members	1
of	1
congress	1
the	1
first	1
lady	1
of	1
the	1
united	1
states	1
and	1
citizens	1
of	1
america	1
tonight	1
as	1
we	1
mark	1
the	1
conclusion	1
of	1
our	1
celebration	1
of	1
black	1
history	1
month	1
we	1
are	1
reminded	1
of	1
our	1
nations	1
path	1
towards	1
civil	1
rights	1
and	1
the	1
work	1
that	1
still	1
remains	1
to	1
be	1
done	1
recent	1
threats	1
targeting	1
jewish	1
community	1
centers	1
and	1
vandalism	1
of	1
jewish	1
cemeteries	1
as	1
well	1
as	1
last	1
weeks	1
shooting	1
in	1
kansas	1
city	1
remind	1
us	1
that	1
while	1
we	1
may	1
be	1
a	1
nation	1
divided	1
on	1
policies	1
we	1
are	1
a	1
country	1
that	1
stands	1
united	1
in	1
condemning	1
hate	1
and	1
evil	1
in	1
all	1
of	1
its	1
very	1
ugly	1
forms	1
each	1
american	1
generation	1
passes	1
the	1
torch	1
of	1
truth	1
liberty	1
and	1
justice	1
in	1
an	1
unbroken	1
chain	1
all	1
the	1
way	1
down	1
to	1
the	1
present	1
that	1
torch	1
is	1
now	1
in	1

### Text -> Mapper -> Reducer

Now we pipe that output to the reducer.

That results in output of the words (the keys) and aggregated counts. Note that these come out in essentially random order.

In [5]:
cat trumpsotu2017.txt | python mapper.py | python reducer.py

voice	2
further	3
missile	1
enrich	1
coordinate	1
loved	3
100	1
task	2
finally	6
launch	1
three	1
greater	4
65	1
australia	1
frank	1
other	8
join	2
sanctions	1
greatly	1
concerns	1
partnerships	1
class	3
around	1
theres	1
respect	4
things	2
hate	1
baltimore	1
thereby	1
means	1
investment	1
drugs	3
fair	5
placed	1
hire	1
footprints	1
mistreated	1
how	3
million	3
prison	1
rebellion	1
in	79
one	14
applause	2
or	7
world	15
china	1
imposing	2
not	26
lawn	1
another	3
spread	1
old	3
simply	1
promised	2
illness	1
chain	1
out	8
vandalism	1
following	1
nonessential	1
trudeau	1
ive	3
as	25
poisoning	1
convicted	1
of	149
crumbling	2
dollars	7
nafta	1
weve	8
align	1
warned	1
too	6
crowley	1
son	1
wants	1
imposed	1
measures	1
ultimately	1
ready	2
middle	5
overcome	1
remember	1
miracles	2
differences	1
mr	2
restart	1
can	11
cooperation	1
declared	1
shifted	1
jewish	2
sheriff	1
france	1
at	19
evil	1
bring	4
majority	1
presidents	1
value	1
workers	6
charter	1
undertaken	1
daring	1
failed	2
small	2
vete

### Text -> Mapper -> Reducer -> Sort alphabetically

We can make it nicer by piping that to `sort`, which will output the keys in alphabetical order.


In [6]:
cat trumpsotu2017.txt | python mapper.py | python reducer.py | sort

1	2
100	1
100th	1
116	1
15	1
17yearold	1
1876	1
20	2
2001	1
2015	1
2016	1
250	2
250th	4
3	1
4000	1
43	2
5	2
5year	1
6	2
60000	1
65	1
8	1
800	1
8—a	1
9	1
911	1
a	94
abandonment	1
ability	1
able	4
about	2
above	1
abraham	1
abroad	1
academy	1
acceptable	1
access	5
accessible	1
accomplish	1
according	2
accounts	1
achieve	3
acknowledge	1
across	10
act	2
action	2
addicted	1
administration	5
admission	1
adopting	1
advance	1
advances	1
advantage	2
advice	1
affordable	1
african	1
after	1
again	2
against	2
against—not	1
against—the	1
agency	1
aggressive	1
ago	1
air	1
airports	1
alexander	1
align	1
all	33
alliance	2
allies	6
allow	4
almost	2
alone	2
along	2
also	9
always	2
am	13
america	27
american	33
americans	15
americas	2
among	1
amounts	1
an	18
and	208
anniversary	2
announced	1
another	3
answered	1
antonin	1
any	2
anyone	2
anywhere	2
appeals	1
applause	2
appoint	1
approval	1
approve	2
approved	1
approximately	1
are	37
arizona	1
around	1
artificially	1
artists	1
as	25
ask	2
asked	2
asking	6
as

### Text -> Mapper -> Reducer -> Sort by frequency

Or, using some of the options of `sort` (telling it to sort by the second column, treat the numbers as numbers, and sort in descending order), we can sort by frequency.


In [7]:
cat trumpsotu2017.txt | python mapper.py | python reducer.py | sort --key 2 --numeric-sort --reverse

the	231
and	208
to	150
of	149
our	116
we	102
a	94
in	79
that	63
is	58
for	57
will	56
have	50
i	38
are	37
with	33
american	33
all	33
they	31
on	30
this	28
be	28
america	27
not	26
by	26
as	25
but	24
you	23
from	23
country	21
us	20
must	20
their	19
at	19
who	18
new	18
its	18
an	18
so	17
it	17
very	16
world	15
was	15
people	15
great	15
americans	15
united	14
one	14
more	14
just	14
tonight	13
should	13
nation	13
has	13
am	13
those	12
my	12
many	12
time	11
states	11
now	11
his	11
citizens	11
can	11
work	10
them	10
every	10
down	10
do	10
across	10
year	9
when	9
want	9
thank	9
jobs	9
health	9
been	9
also	9
weve	8
were	8
than	8
system	8
since	8
out	8
other	8
nations	8
national	8
much	8
immigration	8
he	8
government	8
future	8
congress	8
children	8
years	7
where	7
what	7
right	7
or	7
no	7
long	7
justice	7
home	7
help	7
first	7
dollars	7
create	7
companies	7
believe	7
workers	6
trade	6
too	6
then	6
tax	6
support	6
she	6
same	6
past	6
never	6
need	6
millions	6
members	6
make	6
made	6
life	6
last	6

## We can write our mappers and reducers in any scripting language

We could, for example, write our mapper in R:

In [8]:
cat mapper.R

#!/usr/bin/env Rscript
library(stringr)
library(magrittr)

f <- file("stdin")
open(f)
while(length(line <- readLines(f,n=1)) > 0) {
  splitline <- line %>%
    str_to_lower %>%
    str_replace_all("[[:punct:]]", "") %>%
    str_replace_all("\\s\\s+","\\s")   %>%
    str_split("\\s",simplify=FALSE)     %>%
    unlist
  for (word in splitline) {
    cat(word,"\t",1,"\n")
  }
}


### Text -> R mapper

We get the same result from our R mapper that we got from the Python mapper.


In [9]:
cat trumpsotu2017.txt | Rscript mapper.R

thank 	 1 
you 	 1 
very 	 1 
much 	 1 
mr 	 1 
speaker 	 1 
mr 	 1 
vice 	 1 
president 	 1 
members 	 1 
of 	 1 
congress 	 1 
the 	 1 
first 	 1 
lady 	 1 
of 	 1 
the 	 1 
united 	 1 
states 	 1 
and 	 1 
citizens 	 1 
of 	 1 
america 	 1 
tonight 	 1 
as 	 1 
we 	 1 
mark 	 1 
the 	 1 
conclusion 	 1 
of 	 1 
our 	 1 
celebration 	 1 
of 	 1 
black 	 1 
history 	 1 
month 	 1 
we 	 1 
are 	 1 
reminded 	 1 
of 	 1 
our 	 1 
nations 	 1 
path 	 1 
towards 	 1 
civil 	 1 
rights 	 1 
and 	 1 
the 	 1 
work 	 1 
that 	 1 
still 	 1 
remains 	 1 
to 	 1 
be 	 1 
done 	 1 
recent 	 1 
threats 	 1 
targeting 	 1 
jewish 	 1 
community 	 1 
centers 	 1 
and 	 1 
vandalism 	 1 
of 	 1 
jewish 	 1 
cemeteries 	 1 
as 	 1 
well 	 1 
as 	 1 
last 	 1 
weeks 	 1 
shooting 	 1 
in 	 1 
kansas 	 1 
city 	 1 
remind 	 1 
us 	 1 
that 	 1 
while 	 1 
we 	 1 
may 	 1 
be 	 1 
a 	 1 
nation 	 1 
divided 	 1 
on 	 1 
policies 	 1 
we 	 1 
are 	 1 
a 	 1 
country 	 1 
that 	 1 
stands 	 1 
united 	 1

### Text -> R mapper -> Python reducer -> Bash sort

Since everything is text from the standard input to the standard output, we can combine our R mapper with our Python reducer, and we get the identical final result.


In [10]:
cat trumpsotu2017.txt | Rscript mapper.R | python reducer.py | sort --key 2 --numeric-sort --reverse

the 	231
and 	208
to 	150
of 	149
our 	116
we 	102
a 	94
in 	79
that 	63
is 	58
for 	57
will 	56
have 	50
i 	38
are 	37
with 	33
american 	33
all 	33
they 	31
on 	30
this 	28
be 	28
america 	27
not 	26
by 	26
as 	25
but 	24
you 	23
from 	23
country 	21
us 	20
must 	20
their 	19
at 	19
who 	18
new 	18
its 	18
an 	18
so 	17
it 	17
very 	16
world 	15
was 	15
people 	15
great 	15
americans 	15
united 	14
one 	14
more 	14
just 	14
tonight 	13
should 	13
nation 	13
has 	13
am 	13
those 	12
my 	12
many 	12
time 	11
states 	11
now 	11
his 	11
citizens 	11
can 	11
work 	10
them 	10
every 	10
down 	10
do 	10
across 	10
year 	9
when 	9
want 	9
thank 	9
jobs 	9
health 	9
been 	9
also 	9
weve 	8
were 	8
than 	8
system 	8
since 	8
out 	8
other 	8
nations 	8
national 	8
much 	8
immigration 	8
he 	8
government 	8
future 	8
congress 	8
children 	8
years 	7
where 	7
what 	7
right 	7
or 	7
no 	7
long 	7
justice 	7
home 	7
help 	7
first 	7
dollars 	7
create 	7
companies 	7
believe 	7
workers 	6
trade 	6
t

## To be clear ... we can do this faster if the text is this small

We can do it entirely in one piped bash line, for example (with slightly different results due to the differing definitions of "punctuation").

In [11]:
cat trumpsotu2017.txt | tr '[:punct:]' ' ' | tr 'A-Z' 'a-z' | tr -s ' ' | tr ' ' '\n' | sort | uniq -c | sort -rn

 234 the
 212 and
 168 
 151 to
 149 of
 116 our
 113 we
  96 a
  79 in
  66 that
  59 is
  58 for
  57 will
  51 have
  41 i
  37 are
  35 they
  34 with
  34 all
  33 american
  30 s
  30 on
  29 america
  28 this
  28 be
  27 not
  27 it
  26 by
  25 as
  24 you
  24 but
  23 from
  22 country
  20 must
  19 us
  19 their
  19 new
  19 at
  18 who
  18 one
  18 an
  17 world
  17 so
  16 very
  16 great
  15 was
  15 people
  15 nation
  15 americans
  14 united
  14 more
  14 just
  13 tonight
  13 should
  13 has
  13 am
  12 year
  12 ve
  12 those
  12 states
  12 now
  12 my
  12 many
  12 can
  11 time
  11 his
  11 citizens
  10 work
  10 them
  10 thank
  10 every
  10 down
  10 do
  10 across
   9 when
   9 want
   9 jobs
   9 its
   9 health
   9 government
   9 been
   9 also
   8 than
   8 system
   8 since
   8 out
   8 other
   8 national
   8 much
   8 long
   8 immigration
   8 he
   8 future
   8 congress
   8 children
   7 years
   7 workers
   7 where
   7 what
  